## Required Libraries

In [ ]:
%pip install transformers datasets torch

  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.8 MB 1.5 MB/s eta 0:00:07
   ---- ----------------------------------- 1.3/10.8 MB 3.0 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/10.8 MB 3.0 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/10.8 MB 1.7 MB/s eta 0:00:06
   ------ --------------------------------- 1.8/10.8 MB 1.6 MB/s eta 0:00:06
   ------- -------------------------------- 2.1/10.8 MB 1.6 MB/s eta 0:00:06
   -------- ------------------------------- 2.4/10.8 MB 1.6 MB/s eta 0:00:06
   -------- ------------------------------- 2.4/10.8 MB 1.6 MB/s eta 0:00:06
   --------- ------------------------------ 2.6/10.8 MB 1.3 MB/s eta 0:00:07
   --------- ------------------------------ 2.6/10.8 MB 1.3 MB/s eta 0:00:07
   ---------- -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Dataset PreProcessing

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer


with open(r"C:\Users\Sam\Desktop\Instagram_Caption_GenAI_Project\Instagram_Caption_Generator_UsingGenAI\Instagram_Caption_Dataset.txt", "r", encoding="utf-8") as f:
    poems = [p.strip() for p in f.read().split("\n\n") if len(p.strip().split()) > 10]

dataset = Dataset.from_dict({"text": poems})
dataset = dataset.train_test_split(test_size=0.1)

tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


tokenized_datasets = dataset.map(tokenize_function, batched=True)



c:\Users\Sam\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Sam\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sam\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to

In [ ]:
%pip install hf_xet

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.7 MB 1.2 MB/s eta 0:00:02
   ----------- ---------------------------- 0.8/2.7 MB 1.2 MB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.7 MB 1.2 MB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.7 MB 1.2 MB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.7 MB 1.2 MB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 857.8 kB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 857.8 kB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Fine Tuning GPT2 Model

In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

model = AutoModelForCausalLM.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./poetry-gpt2-finetuned",
    num_train_epochs=1,                  
    per_device_train_batch_size=10,      
    save_steps=1000,
    save_total_limit=1,
    logging_steps=100,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,                         
    max_steps=1500,                       
    dataloader_num_workers=4,            
    gradient_accumulation_steps=1,       
    warmup_steps=50,                     
    logging_dir='./logs',
    report_to=None                      
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

trainer.train()
model.save_pretrained("./poetry-gpt2-finetuned")
tokenizer.save_pretrained("./poetry-gpt2-finetuned")


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
%pip install --upgrade jupyter ipywidgets

^C
Note: you may need to restart the kernel to use updated packages.
